In [3]:
from pymilvus import MilvusClient
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import pickle
import tqdm
import pandas as pd
import requests
import torch
import numpy as np

#########   KEY IN YOUR IP  #########
client = MilvusClient(
    uri="http://192.168.1.111:19530"
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
model.to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
def get_clip_embedding(image_path):
    image = Image.open(image_path)
    inputs = processor(images=image, return_tensors="pt")
    inputs = inputs.to(device)
    outputs = model.get_image_features(**inputs)
    outputs = outputs.cpu()  
    return outputs

In [ ]:
image_path = input('Paste the path of your image here:')
image_embedding = get_clip_embedding(image_path)
image_embedding = image_embedding.detach().numpy()
image_embedding = image_embedding.squeeze()
image_embedding = np.array(image_embedding)

In [ ]:
data = [image_embedding]
res = client.search(
    collection_name="image", # Replace with the actual name of your collection
    data=data,
    limit=1,  # Max. number of search results to return
    search_params={"metric_type": "COSINE", "params": {}}, # Search parameters
    partition_names=["_default"] # Partition names to search in
)

id = int(input('Key in id:'))
xxx = input('Key in name:')

partition_id = res[0][0]['id']
print(partition_id)
data = {"id": id, "vector": image_embedding, "name": xxx}
res = client.insert(
    collection_name="image",
    data=data,
    partition_name="partition"+str(partition_id)
)

In [ ]:
import pickle

class BPlusTreeNode:
    def __init__(self, is_leaf=False):
        self.is_leaf = is_leaf
        self.keys = []
        self.values = []  # This will be a list of (vector, list) tuples for leaves, and empty for internal nodes
        self.children = []

class BPlusTree:
    def __init__(self, order):
        self.order = order
        self.root = BPlusTreeNode(is_leaf=True)

    def search(self, key):
        current_node = self.root
        while not current_node.is_leaf:
            i = 0
            while i < len(current_node.keys) and key >= current_node.keys[i]:
                i += 1
            current_node = current_node.children[i]
        for i in range(len(current_node.keys)):
            if current_node.keys[i] == key:
                return current_node.values[i]
        return None

    def insert(self, key, vector, list_values):
        root = self.root
        if len(root.keys) == (2 * self.order) - 1:
            new_root = BPlusTreeNode()
            new_root.children.append(self.root)
            self.split_child(new_root, 0)
            self.root = new_root
        self._insert_non_full(self.root, key, vector, list_values)

    def _insert_non_full(self, node, key, vector, list_values):
        if node.is_leaf:
            i = 0
            while i < len(node.keys) and key > node.keys[i]:
                i += 1
            node.keys.insert(i, key)
            node.values.insert(i, (vector, list_values))
        else:
            i = 0
            while i < len(node.keys) and key > node.keys[i]:
                i += 1
            if len(node.children[i].keys) == (2 * self.order) - 1:
                self.split_child(node, i)
                if key > node.keys[i]:
                    i += 1
            self._insert_non_full(node.children[i], key, vector, list_values)

    def split_child(self, parent, index):
        order = self.order
        node = parent.children[index]
        new_node = BPlusTreeNode(is_leaf=node.is_leaf)
        parent.keys.insert(index, node.keys[order - 1])
        parent.children.insert(index + 1, new_node)

        new_node.keys = node.keys[order:(2 * order - 1)]
        node.keys = node.keys[:(order - 1)]

        if not node.is_leaf:
            new_node.children = node.children[order:(2 * order)]
            node.children = node.children[:order]

        if node.is_leaf:
            new_node.values = node.values[order - 1:]
            node.values = node.values[:order - 1]
            new_node.children.append(node.children[-1])
            node.children[-1] = new_node

    def serialize(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def deserialize(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)

class DatabaseTable:
    def __init__(self, order):
        self.index = BPlusTree(order)
        self.records = {}

    def insert(self, key, vector, list_values):
        if key in self.records:
            raise ValueError("Duplicate key insertion is not allowed.")
        self.records[key] = (vector, list_values)
        self.index.insert(key, vector, list_values)

    def search(self, key):
        return self.index.search(key)

    def delete(self, key):
        if key in self.records:
            del self.records[key]
            # Implement deletion from B+ tree if necessary
        else:
            raise KeyError("Key not found.")

    def serialize(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def deserialize(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)

# Example usage
db = DatabaseTable(order=3)
db.insert("Alice", [0.1, 0.2, 0.3], ["item1", "item2"])
db.insert("Bob", [0.4, 0.5, 0.6], ["item3", "item4"])
db.insert("Charlie", [0.7, 0.8, 0.9], ["item5", "item6"])

result = db.search("Bob")
print(result)  # Output should be ([0.4, 0.5, 0.6], ['item3', 'item4'])

# Serialize the database
db.serialize("database.pkl")

# Deserialize the database
new_db = DatabaseTable.deserialize("database.pkl")
new_result = new_db.search("Bob")
print(new_result)  # Output should be the same as before
